In [ ]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [10]:
from sklearn.metrics import f1_score, confusion_matrix, make_scorer, accuracy_score, recall_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

import numpy as np
import optuna
import tqdm as notebook_tqdm
from metrics import (
    equality_opportunity_difference,
    predictive_equality_difference,
    metrics
)

In [11]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

#pipeline = Pipeline(
#    steps=[
#        ("preprocessor", preprocessor),
#        ("classifier", LGBMClassifier(n_jobs=-1)),
#    ]
#)

/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [12]:
results = []


In [23]:

directions = ['minimize', 'maximize']
metric_scorer_decorated =  metrics(recall_score, predictive_equality_difference, sensitive_col = 'sex')
for sim in [3]:
    print(sim)
    def objective(trial):

        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.8, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)

        classifier_name = trial.suggest_categorical("classifier", ["logit", "RF", 'GBM','LGBM'])

        if classifier_name == "logit":        
            params = {
                "penalty" : trial.suggest_categorical('logit_penalty', ['l1','l2']),
                "C" : trial.suggest_float('logit_c', 0.001, 10),
                "solver" : 'saga'
                }
            classifier = LogisticRegression(**params)
        
        elif classifier_name =="RF":
            params = {
                'n_estimators': trial.suggest_int("rf_n_estimators", 100, 1000),
                'criterion': trial.suggest_categorical("rf_criterion", ['gini', 'entropy']),
                'max_depth': trial.suggest_int("rf_max_depth", 1, 4),
                'min_samples_split': trial.suggest_float("rf_min_samples_split", 0.01, 1),
                }
            classifier = RandomForestClassifier(**params)

        elif classifier_name =="LGBM":
            params = {
                'n_estimators': trial.suggest_int("lgbm_n_estimators", 20, 10000),
                'num_leaves': trial.suggest_int("lgbm_num_leaves", 10, 1000),
                'max_depth': trial.suggest_int("lgbm_max_depth", 2, 20),
                'min_child_samples': trial.suggest_int("lgbm_min_child_samples", 5, 300),
                'learning_rate': trial.suggest_float('lgbm_learning_rate', 1e-5, 1e-2),
                'boosting_type': trial.suggest_categorical("lgbm_boosting_type", ['goss', 'gbdt'])
                }
            classifier = LGBMClassifier(**params)  
        
        elif classifier_name =="GBM":
            params = {
                'n_estimators': trial.suggest_int("gbm_n_estimators", 100, 1000), 
                'criterion': trial.suggest_categorical("gbm_criterion", ['squared_error', 'friedman_mse']),
                'max_depth': trial.suggest_int("gbm_max_depth", 1, 4),
                'min_samples_split': trial.suggest_int("gbm_min_samples_split", 5, 300),
                }
            classifier = GradientBoostingClassifier(**params)            
        
        else:
            None
        

        pipeline = Pipeline(
            steps=[
                ("preprocessor", preprocessor),
                ("classifier", classifier),
            ]
        )
                
        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_decorated,
                return_train_score=True)

        fair_metric = scores['test_fairness'].mean()
        model_metric = scores['test_model'].mean()

        return fair_metric, model_metric
    
    
    
    study = optuna.create_study(
        directions = directions, 
        pruner = optuna.pruners.SuccessiveHalvingPruner(), 
        sampler = optuna.samplers.TPESampler() 
        )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: ", len(study.trials))
    results.append(study)

[I 2023-05-15 13:14:12,229] A new study created in memory with name: no-name-b57d00b3-8a87-4581-ad75-12e6ff8c41f7


3


[I 2023-05-15 13:14:27,285] Trial 0 finished with values: [0.06621737438525606, 0.6328721243068138] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 531, 'gbm_criterion': 'friedman_mse', 'gbm_max_depth': 2, 'gbm_min_child_samples': 291}. 
[I 2023-05-15 13:14:39,053] Trial 1 finished with values: [0.06790303397633626, 0.6277347682058603] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 409, 'gbm_criterion': 'squared_error', 'gbm_max_depth': 2, 'gbm_min_child_samples': 114}. 
[I 2023-05-15 13:15:02,546] Trial 2 finished with values: [0.0758466666532088, 0.6294578963743847] and parameters: {'classifier': 'LGBM', 'lgbm_n_estimators': 5820, 'lgbm_num_leaves': 678, 'lgbm_max_depth': 9, 'lgbm_min_child_samples': 235, 'lgbm_learning_rate': 0.007754844265395161, 'lgbm_boosting_type': 'gbdt'}. 
/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge


Number of finished trials:  100


In [25]:
import dill
file_name = 'recall-fpr-models-motpe-succesivehalving-100trials-4sim.pkl'
#f1-eod-lgbm-succesivehalving-30trails.pkl
with open(file_name, 'wb') as file:
    dill.dump(results, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "recall-fpr-models-motpe-succesivehalving-100trials-4sim.pkl"


In [19]:
import plotly
optuna.visualization.plot_pareto_front(study, target_names=["FLOPS", "accuracy"])

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.